In [1]:
from keras.preprocessing.image import ImageDataGenerator

img_w, img_h = 150, 150        # 画像をリサイズするときのサイズ
batch_size = 32                # ミニバッチのサイズ

# 訓練データを読み込んで加工するジェネレーターを生成
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,         # RGB値を0～1の範囲に変換
    rotation_range=15,         # 15度の範囲でランダムに回転させる
    zoom_range=0.2,            # ランダムに拡大
    horizontal_flip=True       # 水平方向にランダムに反転
    )

# テストデータを読み込むジェネレーターを生成
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# 訓練データをミニバッチの数だけ生成
train_generator = train_datagen.flow_from_directory(
    'data/train',               # 訓練データのフォルダー
    target_size=(img_w, img_h), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary')        # 二値分類なのでbinaryを指定

# テストデータをミニバッチの数だけ生成
validation_generator = test_datagen.flow_from_directory(
    'data/validation',          # テストデータのフォルダー
    target_size=(img_w, img_h), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary')        # 二値分類なのでbinaryを指定

Using TensorFlow backend.


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [2]:
# 正解ラベルを確認
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'cats': 0, 'dogs': 1}
{'cats': 0, 'dogs': 1}


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras import optimizers

# モデルを構築
model = Sequential()

# （第1層）畳み込み層
model.add(
    Conv2D(
        filters=32,                # フィルターの数は32
        kernel_size=(3, 3),        # 3×3のフィルターを使用
        input_shape=(150, 150, 3), # 入力データの形状
        padding='same',            # ゼロパディングを行う
        activation='relu'          # 活性化関数はReLU
        ))
# （第2層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# （第3層）畳み込み層
model.add(
    Conv2D(
        filters=32,            # フィルターの数は32
        kernel_size=(3, 3),    # 3×3のフィルターを使用
        activation='relu'      # 活性化関数はReLU
        ))
# （第4層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# （第5層）畳み込み層
model.add(
    Conv2D(filters=64,         # フィルターの数は64
           kernel_size=(3, 3), # 3×3のフィルターを使用
           activation='relu'   # 活性化関数はReLU
          ))
# （第6層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト25％
model.add(Dropout(0.25))

# 出力層への入力を4階テンソルから2階テンソルに変換する
model.add(Flatten())

# （第7層）全結合層
model.add(
    Dense(64,                  # ニューロン数は64
          activation='relu'))  # 活性化関数はReLU
# ドロップアウト50％
model.add(Dropout(0.5))

# （第8層）出力層
model.add(
    Dense(
        1,                     # ニューロン数は1個
        activation='sigmoid')) # 活性化関数はsigmoid

# モデルのコンパイル
model.compile(
    loss='binary_crossentropy', # バイナリ用の交差エントロピー誤差
    metrics=['accuracy'],       # 学習評価として正解率を指定
    # Adamアルゴリズムで最適化
    optimizer=optimizers.Adam(),
)

# モデルのサマリを表示
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 73, 73, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [4]:
epochs = 60             # 学習回数
batch_size = batch_size # 設定済みのミニバッチのサイズ

# 学習を行う
history = model.fit_generator(
    train_generator,    # 訓練データ
    epochs=epochs,      # 学習回数
    verbose=1,          # 学習の進捗状況を出力する
    # テストデータ
    validation_data=validation_generator,
)

Epoch 1/60
63/63 [==============================] - 74s 1s/step - loss: 0.7349 - acc: 0.5074 - val_loss: 0.6930 - val_acc: 0.5550
Epoch 2/60
63/63 [==============================] - 70s 1s/step - loss: 0.6907 - acc: 0.5213 - val_loss: 0.6894 - val_acc: 0.5775
Epoch 3/60
63/63 [==============================] - 68s 1s/step - loss: 0.6884 - acc: 0.5516 - val_loss: 0.6931 - val_acc: 0.5100
Epoch 4/60
63/63 [==============================] - 68s 1s/step - loss: 0.6827 - acc: 0.5566 - val_loss: 0.6741 - val_acc: 0.5537
Epoch 5/60
63/63 [==============================] - 68s 1s/step - loss: 0.6787 - acc: 0.5580 - val_loss: 0.6752 - val_acc: 0.6050
Epoch 6/60
63/63 [==============================] - 72s 1s/step - loss: 0.6615 - acc: 0.5888 - val_loss: 0.6536 - val_acc: 0.5962
Epoch 7/60
63/63 [==============================] - 70s 1s/step - loss: 0.6478 - acc: 0.6146 - val_loss: 0.6471 - val_acc: 0.6025
Epoch 8/60
63/63 [==============================] - 70s 1s/step - loss: 0.6479 - acc: 0.62